<a href="https://colab.research.google.com/github/samitha278/miniVGG/blob/main/build2_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader


from dataclasses import dataclass

## Get Data

In [2]:
batch_size = 16

In [3]:
transform_train = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform_train)
val_dataset = datasets.MNIST(root='data', train=False, download=True, transform=transform_train)


train_data = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 62.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.65MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.59MB/s]


In [4]:
image ,label  = next(iter(train_data))
image.shape

torch.Size([16, 1, 28, 28])

In [6]:
flat = nn.Flatten()

flat(image).shape

torch.Size([16, 784])

## Model Implementation

In [21]:
@dataclass
class Config():
    in_ch : int = 1
    n_embd : int = 36
    n_class : int = 10







class minVGG(nn.Module):

    def __init__(self,config):
        super().__init__()



        self.conv_1 = nn.Conv2d(config.in_ch, 2*config.in_ch,5)
        self.pooling_1 = nn.MaxPool2d(3)

        self.conv_2 = nn.Conv2d(2* config.in_ch, 4*config.in_ch,3)
        self.pooling_2 = nn.MaxPool2d(2)

        self.relu = nn.ReLU()

        self.flatten = nn.Flatten()

        self.mlp = MLP(config)


    def forward(self,x,targets = None):

        B,C,H,W = x.shape

        #1
        x = self.conv_1(x)
        x = self.relu(x)
        x = self.pooling_1(x)

        #2
        x = self.conv_2(x)
        x = self.relu(x)
        x = self.pooling_2(x)

        x = self.flatten(x)

        logits = self.mlp(x)

        loss = None
        if targets is not None:
            return logits , F.cross_entropy(logits,targets)

        return logits







class MLP(nn.Module):


    def __init__(self,config):
        super().__init__()

        self.layer = nn.Linear(config.n_embd,4*config.n_embd)
        self.relu = nn.ReLU()
        self.proj = nn.Linear(4*config.n_embd,config.n_class)


    def forward(self,x):

        x = self.layer(x)
        x = self.relu(x)
        x = self.proj(x)

        return x






In [23]:
config = Config()
cnn = minVGG(config)
cnn(image).shape

torch.Size([16, 10])